In [1]:
import numpy as np
import pandas as pd
import os
from os.path import join
import csv


In [2]:
def get_classifier_subdirs(a_dir, only=''):
    if only=='':
        return [os.path.join(a_dir, name) for name in os.listdir(a_dir) 
                if os.path.isdir(os.path.join(a_dir, name)) and ('_k_5_' in name)]


In [3]:
def get_immediate_subdirs(a_dir, only=''):
    if only=='':
        return [os.path.join(a_dir, name) for name in os.listdir(a_dir) 
                if os.path.isdir(os.path.join(a_dir, name)) and name.endswith('_l')]


In [4]:
def get_num_det(cm,attack_name):
    #print(cm.loc[cm['Unnamed: 0']==attack_name])
    if attack_name not in cm:
        return 0
    
    row = cm.loc[cm['Unnamed: 0']==attack_name,]    
    c = row[attack_name]
    return c.values[0]

In [13]:
def print_absolute_recall(cm,label_names,filename):
    eps = 0.000000000005
    K =5
    df = pd.read_csv('/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l/label_dist.csv',names=['Label','Count'],header=None)
    all_label_names = df['Label'].values
    with open(filename,'w') as f:
        w = csv.writer(f)
        w.writerow(['Label','Rc'])

        m_rc = 0
        index=1
        for i,label in enumerate(all_label_names):

            if label in label_names:
                tp = get_num_det(cm,label)
                index+=1
            else:
                tp = 0
            total_count = df[df['Label']==label]['Count'].values[0]
            total_test_count = total_count*1./K
            rc = tp/(total_test_count)

            w.writerow(["{}".format(label),'{:.4f}'.format(rc)])
            
            #if label != 'Benign':
            m_rc += rc

        n = len(label_names)
        w.writerow(["{}".format('Macro avg'),'{:.4f}'.format(m_rc/n)])


In [14]:
    df = pd.read_csv('/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l/label_dist.csv',names=['Label','Count'],header=None)
    all_label_names = df['Label'].values 
    all_label_names

array(['Benign', 'Brute Force-Web', 'Brute Force-XSS',
       'DDoS attacks-LOIC-HTTP', 'DDoS-HOIC', 'DDoS-LOIC-UDP',
       'DoS-GoldenEye', 'DoS-Hulk', 'DoS-SlowHTTPTest', 'DoS-Slowloris',
       'FTP-BruteForce', 'Infiltration', 'SQL Injection',
       'SSH-BruteForce'], dtype=object)

In [17]:
root = '/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/'
data_dirs = get_immediate_subdirs(root)
print(data_dirs)
for data_dir in data_dirs:
    print('-----------------------------------------')
    print(data_dir)
    classifier_dirs = get_classifier_subdirs(data_dir)
    print("classifier dirs")
    print(classifier_dirs)
    for classifier_dir in classifier_dirs:
        if 'bs_128' not in classifier_dir:
            continue        
        print('*************************************')
        print(classifier_dir)
        cm_filename = join(classifier_dir,'cm_nonnorm_fold_avg.csv')
        cm = pd.read_csv(cm_filename)
        rc_filename=  cm_filename.replace('cm_nonnorm_fold_avg.csv','absolute_recall.csv')
        print_absolute_recall(cm,all_label_names,rc_filename)        

['/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l']
-----------------------------------------
/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l
classifier dirs
['/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l/cnn_k_5_bs_5120', '/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l/cnn_k_5_128', '/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l/cnn_k_5_bs_16_w', '/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l/cnn_k_5_bs_5120_w', '/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l/cnn_k_5_bs_16', '/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l/cnn_k_5_w_optim_Adam_lr_0.001_reg_1e-05_bs_128', '/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l/cnn_k_5_bs_64_w']
*************************************
/home/juma/data/net_intrusion/CIC-IDS-2018/without_sampling/whole_l/cnn_k_5_w_optim_Adam_lr_0.001_reg_1e-05_bs_128


In [ ]:
cm_filename = join('/home/juma/data/net_intrusion/CIC-IDS-2018/CSVs_mem_SF_r_1/SR_1/ffs_(4,8,32)_l/forest_k_5','cm_nonnorm_fold_avg.csv')